In [174]:
import pandas as pd
import numpy as np
import pprint as pp
import re, string
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import praw

import credentials as creds

In [175]:
# https://praw.readthedocs.io/en/latest/getting_started/
# https://www.reddit.com/dev/api/

In [176]:
CLIENT_ID = creds.client_id()
CLIENT_SECRET_KEY = creds.client_secret_key()


r = praw.Reddit(client_id = CLIENT_ID,
                client_secret = CLIENT_SECRET_KEY,
                user_agent = 'RedditorMatch')

print(r.read_only)

True


In [178]:
user = r.redditor("BottomsMU")

comments = user.comments.top(limit = 3)
for i in comments:
    print(i.body[0:5])
    print(i.author_flair_text)
    print(i.subreddit_name_prefixed)
    print(i.score)
    print(i.ups)
    print(i.downs)
    print(i.controversiality)
    print(i.created_utc)
    print(i.gilded)
    print(i.over_18)
    print(i.link_permalink)
    print("================================")

"Rand
None
r/trailerparkboys
92
92
0
0
1411334176.0
0
False
https://www.reddit.com/r/trailerparkboys/comments/2h28wb/your_favourite_jroc_one_liners/
I hat
None
r/google
56
56
0
0
1405442311.0
0
False
https://www.reddit.com/r/google/comments/2arpge/its_a_beautiful_day_in_the_fiberhood/
I for
None
r/mizzou
43
43
0
0
1447272500.0
0
False
https://www.reddit.com/r/mizzou/comments/3sg2ib/is_the_entire_mizzou_protest_based_on_lies/


In [9]:
def find_similar(matrix, index, top_n = 5):
    cosine_similarities = linear_kernel(matrix[index: index + 1], matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [10]:
def findMatches(your_username, subreddit):
    corpus = []
    corpusDf = scrapeCommentsFromSubreddit(subreddit)
    corpusDf["Comments"].apply(lambda row: corpus.append(str(row)))
    
    your_comments = getUserComments(your_username)
    your_comments = ".".join(your_comments)
    
    corpus.insert(0, your_comments)
    
    tf = TfidfVectorizer(analyzer = "word", 
                            ngram_range = (1, 3),
                            min_df = 0, 
                            stop_words = "english")
    
    matrix = tf.fit_transform(corpus)
    results = []
    
    for index, score in find_similar(matrix, 0):        
        index = index - 1 # because we prepended our comments onto the corpus, the index number was shifted by 1.
        user = corpusDf.iloc[index, 0]
        results.append(user)
        print("Score:", score, "| Username:", user)
        print("=========================================================")
        
    return(results)

In [11]:
matches = findMatches(your_username = "ohai123456789", subreddit = "datascience")

KeyboardInterrupt: 

In [ ]:
len(matches)

In [ ]:
matches[0:10]

In [ ]:
def whyMatch(your_username, their_usernames):
    # find subreddit subscriptions
    # find common words
    return